# TSE TAKEHOME DATASET
## Creating a Data Analysis Assistant


In [130]:
#Package Imports

from openai import OpenAI # type: ignore
import pandas as pd  # type: ignore
import dotenv # type: ignore
import os
import json
from openai.types.beta.threads.text_content_block import TextContentBlock # type: ignore



dotenv.load_dotenv() # type: ignore


print(os.environ)
print(os.getenv("OPENAI_API_KEY"))


environ({'COMMAND_MODE': 'unix2003', 'HOME': '/Users/bytedance', 'LOGNAME': 'bytedance', 'MallocNanoZone': '0', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'PATH': '/Users/bytedance/Documents/python-env/.venv/bin:/Library/Frameworks/Python.framework/Versions/3.11/bin:/opt/anaconda3/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/sbin:/opt/puppetlabs/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin', 'PWD': '/', 'SHELL': '/bin/bash', 'SHLVL': '2', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.IAi0IW9bfk/Listeners', 'TMPDIR': '/var/folders/fn/vkcznhxx1sv7ym1hswkzbs680000gn/T/', 'USER': 'bytedance', 'VSCODE_AMD_ENTRYPOINT': 'vs/workbench/api/node/extensionHostProcess', 'VSCODE_CODE_CACHE_PATH': '/Users/bytedance/Library/Application Support/Code/CachedData/dc96b837cf6b

#### Testing and Setting up OpenAI Client 

In [15]:
#Loading the client with chat completion example

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

In [20]:
print(completion.choices[0].message.content)

In the world of code, a tale unfolds,
Where recursion's beauty brightly glows.
A function calling itself with grace,
In an endless, looping embrace.

Like a mirror reflecting in endless sight,
Recursion echoes through the silent night.
Each step repeated with wondrous art,
Unraveling logic, touching the heart.

A loop of patterns, a dance of rhyme,
Unfolding mysteries, one step at a time.
Breaking problems into pieces small,
Recursive magic, enchanting all.

So remember, in the coder's art,
Recursion plays a special part.
A concept deep, a concept grand,
In the programmer's enchanted land.


### Set-up Assistant Client with Code Interpreter enabled on input file

In [214]:
#Read input file for assistant / thread
file = client.files.create(
  file=open("tse_takehome_dataset.csv", "rb"),
  purpose='assistants'
)

In [234]:
#Create the Assistant 

assistant = client.beta.assistants.create(
  instructions="You are a personal math tutor. When asked a math question, write and run code to answer the question.",
  model="gpt-3.5-turbo",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

In [220]:
#Create the thread for the assistant 


thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "explain from the file attached why a city is each person's favorite city",
    }
  ]
)


In [221]:
#Create the run for the thread messages 

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
  #instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [222]:
#Print run messages without streaming 
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_iRAwjjJbKpaqSKWujljZfHn9', assistant_id='asst_j3uE4fq9712iDZwbzCvRO3Mf', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='From the analysis of the "favourite_city_and_why" column in the data, we can see the reasons provided by each person for why a specific city is their favorite. Here are the reasons for some individuals:\n\n- **James Padilla**: London, for its historical landmarks and diverse cultural scene.\n- **Eric Rogers**: Paris, for its beautiful architecture.\n- **Christopher King**: Tokyo, for its unique blend of traditional and modern.\n- **Tina Escobar**: New York, because of its vibrant city life.\n- **Joshua Lewis**: Paris, for its beautiful architecture.\n- **Samantha Becker**: Sydney, for its stunning harbour.\n- **Jesse Crawford**: Sydney, for its stunning harbour.\n- **Ronald Jones**: Sydney, for its stunning harbour.\n- **Michael Ali**: London, for its historical landmarks and 

In [223]:
#Pretty print Assistant messages. Similar to text output on the Playground and Recommended ChatGPT interface

saved_data = []


for i in messages.data:
    message_dict = {
    "id": i.id,
    "assistant_id": i.assistant_id,
    "thread_id": i.thread_id,
    "content":i.content,
    "role": i.role
    # Add more attributes as needed
    }
    saved_data.append(message_dict)

for j in range(len(saved_data) - 1, -1, -1):
    if saved_data[j]["role"] =='assistant':
        print(saved_data[j]["content"][0].text.value)

To explain why a city is each person's favorite city, we need to look into the contents of the uploaded file to understand the criteria or preferences that determine each person's favorite city.

Let's start by examining the contents of the file to see how the favorite city is determined for each person.
The file contains information about several individuals, including their names, company names, favorite memories, favorite cities, favorite foods, occupations, job descriptions, relevant job experiences, and growth plans. 

To determine why a city is each person's favorite city, we need to look at the "favourite_city_and_why" column in the file. This column provides insight into why each person considers a specific city to be their favorite.

Let's analyze the contents of the "favourite_city_and_why" column to extract the reasons why each person has chosen a particular city as their favorite.
From the analysis of the "favourite_city_and_why" column in the data, we can see the reasons p

#### Dataset Troubleshooting using Pandas

Investigate the data inputs using Pandas

In [225]:
df = pd.read_csv('tse_takehome_dataset.csv')
df.columns


Index(['date', 'name', 'company_name', 'description_of_company',
       'favourite_memory', 'favourite_city_and_why', 'favourite_food_and_why',
       'occupation', 'description_of_job', 'experience_relevant_to_job',
       'growth_plan'],
      dtype='object')

In [226]:
df1=df[['name','favourite_city_and_why','favourite_food_and_why']]

In [227]:
# Keeping pandas from truncating long strings
pd.set_option('display.max_colwidth', 100000)
df1.head(20)

,name,favourite_city_and_why,favourite_food_and_why
0,James Padilla,"London, for its historical landmarks and diverse cultural scene. for its historical landmarks and diverse cultural scene. Additionally, London has hosted the Summer Olympics three times: in 1908, 1948, and 2012.","Pizza, because it's versatile, delicious, and brings people together."
1,Eric Rogers,"Paris, for its beautiful architecture for its beautiful architecture Additionally, The Eiffel Tower was supposed to be a temporary installation, intended to stand for 20 years after being constructed for the 1889 World Fair.","Chocolate, because it's comforting, decadent, and a sweet treat for any occasion."
2,Christopher King,"Tokyo, for its unique blend of traditional and modern for its unique blend of traditional and modern Additionally, It's considered one of the world's most important and powerful global cities.","Sushi, for its fresh flavors and artful presentation."
3,Tina Escobar,"New York, because of its vibrant city life and diversity. because of its vibrant city life and diversity. Additionally, It's home to the largest metropolitan zoo in the US.","Pasta, for its comfort, versatility in dishes, and rich flavors."
4,Joshua Lewis,"Paris, for its beautiful architecture for its beautiful architecture Additionally, Paris is known as the 'City of Light', originally because of its leading role during the Age of Enlightenment and more literally because Paris was one of the first large European cities to use gas street lighting on a grand scale on its boulevards and monuments.","Pizza, because it's versatile, delicious, and brings people together."
5,Samantha Becker,"Sydney, for its stunning harbour for its stunning harbour Additionally, Sydney Harbour Bridge is the world's largest steel arch bridge.","Chocolate, because it's comforting, decadent, and a sweet treat for any occasion."
6,Jesse Crawford,"Sydney, for its stunning harbour for its stunning harbour Additionally, The Sydney Opera House design was inspired by orange segments.","Pasta, for its comfort, versatility in dishes, and rich flavors."
7,Ronald Jones,"Sydney, for its stunning harbour for its stunning harbour Additionally, The Sydney Opera House design was inspired by orange segments.","Pizza, because it's versatile, delicious, and brings people together."
8,Michael Ali,"London, for its historical landmarks and diverse cultural scene. for its historical landmarks and diverse cultural scene. Additionally, London has hosted the Summer Olympics three times: in 1908, 1948, and 2012.","Tacos, for their flavor-packed bites and customizable fillings."
9,Kristy Alvarez,"New York, because of its vibrant city life and diversity. because of its vibrant city life and diversity. Additionally, It's home to the largest metropolitan zoo in the US.","Sushi, for its fresh flavors and artful presentation."


#### Issue identified:
 "favourite_city_and_why" needs to be clean-up and formatted similar to "favourite_food_and_why"

Steps:
1. Extract true reason for favorite city for each individual by spliting the string on "Additionally," and picking the last element in the array
2. Extract the city from the string by splitting the string on "," and picking the first element in the array
3. Replace the "favourite_city_and_why" column with the cleaned up city and reason
4. Create a new csv named "tse_takehome_dataset_fixed.csv" with the cleaned up data for favourite_city_and_why
5. Rerun Assistant and model output using results from "tse_takehome_dataset_fixed_results.csv"

In [228]:
import re

# Clean up "favourite_city_and_why" column in the dataset
text = df["favourite_city_and_why"]
list_updated = []

for i in text:
    why_favorite_city= i.split('Additionally,')[1]
    city= i.split('Additionally,')[0].split(',')[0]
    favorite_city_and_why= city + ', because' + why_favorite_city

    # Split the string using comma and period as delimiters
    #result = re.split(r'[.,]', i)

    # Remove empty strings from the list
    # result = [x for x in result if x !='']
    print(favorite_city_and_why)
    list_updated.append(favorite_city_and_why)

London, because London has hosted the Summer Olympics three times: in 1908, 1948, and 2012.
Paris, because The Eiffel Tower was supposed to be a temporary installation, intended to stand for 20 years after being constructed for the 1889 World Fair.
Tokyo, because It's considered one of the world's most important and powerful global cities.
New York, because It's home to the largest metropolitan zoo in the US.
Paris, because Paris is known as the 'City of Light', originally because of its leading role during the Age of Enlightenment and more literally because Paris was one of the first large European cities to use gas street lighting on a grand scale on its boulevards and monuments.
Sydney, because Sydney Harbour Bridge is the world's largest steel arch bridge.
Sydney, because The Sydney Opera House design was inspired by orange segments.
Sydney, because The Sydney Opera House design was inspired by orange segments.
London, because London has hosted the Summer Olympics three times: in 1

In [229]:
#update dataset dataframe with new values
df["favourite_city_and_why"]= list_updated

In [230]:
#write to data to csv file
df.to_csv('tse_takehome_dataset_fixed.csv', index=False)

#### Re-run Assistant Client with Code Interpretor and check new model output to prompt

In [260]:
#Read input file for assistant / thread
file = client.files.create(
  file=open("tse_takehome_dataset_fixed.csv", "rb"),
  purpose='assistants'
)

In [250]:
#Create the Assistant 

assistant = client.beta.assistants.create(
  #instructions="Analyze the data and provide insights and answers to the prompts",
  model="gpt-3.5-turbo",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

In [261]:
#Create the thread for the assistant 


thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "explain from the file attached why a city is each person's favorite city",
    }
  ]
)


In [262]:
#Create the run for the thread messages 

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
  #instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [263]:
#Print run messages without streaming 
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_g6jGzAes0DHckWH4YSEcyDlo', assistant_id='asst_12lWyIFyKhTW1xTsVPEjlPZH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Here is the information on why each person's favorite city is their favorite:\n\n- **James Padilla:** London, because London has hosted the Summer Olympics three times: in 1908, 1948, and 2012.\n- **Eric Rogers:** Paris, because The Eiffel Tower was supposed to be a temporary installation, intended to stand for 20 years after being constructed for the 1889 World Fair.\n- **Christopher King:** Tokyo, because it's considered one of the world's most important and powerful global cities.\n- **Tina Escobar:** New York, because it's home to the largest metropolitan zoo in the US.\n- **Joshua Lewis:** Paris, because Paris is known as the 'City of Light', originally because of its leading role during the Age of Enlightenment and more literally because Paris was one of the first large 

In [264]:
#Pretty print Assistant messages. Similar to text output on the Playground and Recommended ChatGPT interface

saved_data = []


for i in messages.data:
    message_dict = {
    "id": i.id,
    "assistant_id": i.assistant_id,
    "thread_id": i.thread_id,
    "content":i.content,
    "role": i.role
    # Add more attributes as needed
    }
    saved_data.append(message_dict)

for j in range(len(saved_data) - 1, -1, -1):
    if saved_data[j]["role"] =='assistant':
        print(saved_data[j]["content"][0].text.value)

To help you analyze the file and determine why a city is each person's favorite city, I will first need to read the contents of the file to understand its structure. Let's start by loading the file and examining its contents.
The file contains data in a CSV format, with each line representing an individual's information. Here are the fields present in the file:

1. date
2. name
3. company_name
4. description_of_company
5. favourite_memory
6. favourite_city_and_why
7. favourite_food_and_why
8. occupation
9. description_of_job
10. experience_relevant_to_job
11. growth_plan

To determine why a city is each person's favorite city, we will focus on the "favourite_city_and_why" field. We will extract this information along with the person's name from each line in the file.

Let's extract this information and analyze why each city is each person's favorite.
Here is the information on why each person's favorite city is their favorite:

- **James Padilla:** London, because London has hosted the